In [0]:
df_musicas = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/lucas.paiola@estudante.ufscar.br/dataset.csv")

In [0]:
df_musicas = df_musicas.limit(5000)

Pré-processamento dos dados originados do Kaggle para posterior inserção no Neo4j Sandbox
- Observações:
- - Apenas o primeiro artista (principal) de cada música foi considerado como "autor". Colaborações entre mais de um artista não são considerados.

In [0]:
from pyspark.sql.functions import col, lower, regexp_replace, when, split, trim
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#utilizar o numero serial como track_id e remover o track id original (não iremos usar)
df_musicas = df_musicas.drop("track_id")
df_musicas = df_musicas.withColumnRenamed("_c0", "track_id")
df_musicas = df_musicas.withColumnRenamed("key", "chave")
df_musicas = df_musicas.withColumnRenamed("artists", "artist")

#limpar as colunas que tem caracteres especiais e letras maiúsculas, entre outros
colunas_a_limpar = ["artist", "album_name", "track_name"]

for coluna in colunas_a_limpar:
    df_musicas = df_musicas.withColumn(
        coluna,
        lower(regexp_replace(col(coluna), "[^\w\s;]", ""))
    )

#trocar false por 0 e true por 1 na coluna "explicit"
df_musicas = df_musicas.withColumn("explicit", when(col("explicit") == True, 1).otherwise(0))

#considerar apenas o artista principal da música, mesmo em colaborações
df_musicas = df_musicas.withColumn("artist", split(col("artist"), ";")[0]).withColumn("artist", trim(col("artist")))

#remover as linhas que não possuam artista ou nome da música
df_musicas = df_musicas.dropna(subset=["artist", "track_name"])


# Definir a janela para calcular a média de popularidade por álbum
window_spec = Window.partitionBy("album_name")
df_musicas = df_musicas.withColumn("album_popularity", F.round(F.avg("popularity").over(window_spec), 1))

#Preencher com 0 as colunas que serão usadas e possuem valores nulos
df_musicas = df_musicas.fillna(0, subset=["popularity", "danceability", "energy", "chave", "loudness",
                               "speechiness", "acousticness", "instrumentalness", "liveness", "time_signature"])

# Remover linhas duplicadas baseado no artista e nome da música
df_musicas = df_musicas.dropDuplicates(["artist", "track_name"])

display(df_musicas)

track_id,artist,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,chave,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,album_popularity
2118,3 doors down,rock,here without you,0,238560,0,0.556,0.545,10,-6.768,0,0.0255,0.0635,0.0,0.124,0.193,143.991,4,alt-rock,0.0
2810,3 doors down,seventeen days,here without you acoustic version,64,232013,0,0.551,0.574,10,-6.265,0,0.0263,0.0203,0.0,0.186,0.308,143.734,4,alt-rock,64.0
2113,3 doors down,emo christmas 2022,its not my time acoustic,0,238373,0,0.674,0.58,0,-8.081,1,0.0312,0.311,2.41e-05,0.198,0.305,128.003,4,alt-rock,0.0
2120,3 doors down,fishing music,kryptonite,0,234626,0,0.544,0.848,11,-6.804,0,0.0288,0.00395,5.22e-05,0.134,0.489,99.039,4,alt-rock,0.3
264,a fine frenzy,one cell in the sea,almost lover,57,268800,0,0.549,0.24,0,-7.778,1,0.0338,0.947,0.0,0.106,0.167,119.58,4,acoustic,57.0
932,a great big world,is there anybody out there track by track commentary,already home,44,230186,0,0.602,0.404,2,-7.344,1,0.0272,0.31,0.0,0.14,0.243,83.054,4,acoustic,55.0
431,a great big world,particles,fall on me,50,262820,0,0.393,0.268,9,-6.741,1,0.0385,0.697,0.0,0.11,0.225,110.456,4,acoustic,50.0
709,a great big world,when the morning comes,hold each other feat futuristic,47,216040,0,0.61,0.796,10,-5.431,1,0.0656,0.0365,0.0,0.274,0.51,159.944,4,acoustic,54.5
393,a great big world,is there anybody out there track by track commentary,i dont wanna love somebody else,49,163960,0,0.495,0.22,5,-11.16,1,0.0355,0.847,1.4e-06,0.0957,0.345,126.618,4,acoustic,55.0
76,a great big world,when the morning comes,kaleidoscope,62,229320,0,0.709,0.913,7,-5.148,1,0.0748,0.0182,0.0,0.167,0.519,108.024,4,acoustic,54.5


Inserção dos dados limpos no Neo4j Sandbox
- Observação: Os dados para conexão com a instância do Neo4j Sandbox irão mudar de acordo com os dados necessários (cada instância tem url e senhas distintos)

In [0]:
#Informações necessárias - mudar de acordo com as informações da instância do neo4j
bolt = "bolt://54.174.159.156:7687"
password = "composition-absence-allegations"

# Relacionamentos

# Faixa - Artista (artista grava faixa)
(
    df_musicas.write 
    .mode("Overwrite")
    # Conexão com o neo4j
    .format("org.neo4j.spark.DataSource")
    .option("url", bolt)
    .option("authentication.type", "basic")
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password", password)
    # Define o nome do relacionamento
    .option("relationship", "GRAVOU")
    # Estrategia 'keys' sao obrigatorias para o modo 'overwrite'
    .option("relationship.save.strategy", "keys")
    # Reescreve os nós se ja existirem e define a label como Artista
    .option("relationship.source.save.mode", "Overwrite")
    .option("relationship.source.labels", ":Artista")
    # Define as propriedades do nó de artista
    .option(
        "relationship.source.node.properties", "artist:nome"
    )
    .option("relationship.source.node.keys", "artist:nome")
    # Reescreve os nós se ja existirem e define a label como Faixa
    .option("relationship.target.save.mode", "Overwrite")
    .option("relationship.target.labels", ":Faixa")
    # Define as propriedades do nó de faixa
    .option("relationship.target.node.properties", "track_name:nome, popularity, danceability, energy, valence")
    .option("relationship.target.node.keys", "track_id:id")
    .save()
)

# Faixa - Album (faixa pertence a album)
(
    df_musicas.write
    .mode("Overwrite")
    # Conexão com o neo4j
    .format("org.neo4j.spark.DataSource")
    .option("url", bolt)
    .option("authentication.type", "basic")
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password", password)
    # Define o nome do relacionamento
    .option("relationship", "PERTENCE_A")
    .option("relationship.save.strategy", "keys")
    # Reescreve os nós se ja existirem e define a label como Faixa
    .option("relationship.source.save.mode", "Overwrite")
    .option("relationship.source.labels", ":Faixa")
    # Define as propriedades do nó de faixa
    .option(
        "relationship.source.node.properties", "track_name:nome, popularity, danceability, energy, valence"
    )
    .option("relationship.source.node.keys", "track_id:id")
    # Reescreve os nós se ja existirem e define a label como Album
    .option("relationship.target.save.mode", "Overwrite")
    .option("relationship.target.labels", ":Album")
    # Define as propriedades do nó de album
    .option("relationship.target.node.properties", "album_name")
    .option("relationship.target.node.keys", "album_name")
    .save()
)

# Album - Artista (album feito por artista)
(
    df_musicas.write
    .mode("Overwrite")
    # Conexão com o neo4j
    .format("org.neo4j.spark.DataSource")
    .option("url", bolt)
    .option("authentication.type", "basic")
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password", password)
    # Define o nome do relacionamento
    .option("relationship", "FEITO_POR")
    .option("relationship.save.strategy", "keys")
    # Reescreve os nós se ja existirem e define a label como Album
    .option("relationship.source.save.mode", "Overwrite")
    .option("relationship.source.labels", ":Album")
    # Define as propriedades do nó de album
    .option(
        "relationship.source.node.properties", "album_name"
    )
    .option("relationship.source.node.keys", "album_name")
    # Reescreve os nós se ja existirem e define a label como Artista
    .option("relationship.target.save.mode", "Overwrite")
    .option("relationship.target.labels", ":Artista")
    # Define as propriedades do nó de artista
    .option("relationship.target.node.properties", "artist:nome")
    .option("relationship.target.node.keys", "artist:nome")
    .save()
)